## Combine multiple Bracken output files for a statistical comparison of marker taxa using LEfSe

In [24]:
import pandas as pd
import os as os
import glob as glob
import numpy as np

In [25]:
os.chdir("/Users/nastassia.patin/Desktop/Projects/Norovirus/kraken2")

### First extract only the taxon names ('name') and 'new_est_reads' columns from the Bracken raw output

In [ ]:
### I ran this loop in bash
!for f in *fa.bracken; do 
    n=`echo $f | cut -d . -f 1` && 
    !cat $f | !awk -F '\t' '{ print $1"\t"$6 }' > $n'_lefse.csv'; 
done

### Then combine these .csv outputs and merge on taxon name

In [26]:
dfs = []
for file in glob.glob("*lefse.csv"):
    df = pd.read_csv(file, sep='\t', header=0)
    a, b = file.split('.')
    c, d, e = a.split('_')
    name = c + '_' + d
    names=['Taxon', name]
    df.columns=names
    dfs.append(df)

In [27]:
dfs

[                                Taxon    38_1
 0            Bacteroides sp. CACC 737  669996
 1                Bacteroides sp. A1C1  578055
 2             Bacteroides sp. HF-5141   67049
 3             Bacteroides sp. HF-5287   44555
 4             Bacteroides sp. CBA7301   28665
 ...                               ...     ...
 2186    Faecalibacterium virus Brigit      79
 2187  Faecalibacterium virus Toutatis      44
 2188             Salmonella phage SW9     422
 2189      Faecalibacterium virus Lugh      43
 2190     Staphylococcus virus MCE2014      60
 
 [2191 rows x 2 columns],
                                  Taxon     41_1
 0                Bacteroides uniformis  2447623
 1             Bacteroides sp. CACC 737   472113
 2                 Bacteroides sp. A1C1   492398
 3              Bacteroides sp. HF-5141    79152
 4                  Bacteroides sp. M10    68216
 ...                                ...      ...
 2378     Faecalibacterium virus Oengus      122
 2379           

In [29]:
dfs_merged = reduce(lambda  left,right: pd.merge(left,right,on='Taxon',how='outer'), dfs)
dfs_merged = dfs_merged.replace(np.nan,0)
dfs_merged.head()

,Taxon,38_1,41_1,28_1,13_1,4_1,37_1,15_1,49_1,36_1
0,Bacteroides sp. CACC 737,669996.0,472113.0,27171.0,283416.0,316573.0,146508.0,85680.0,192612.0,292567.0
1,Bacteroides sp. A1C1,578055.0,492398.0,18869.0,531038.0,602402.0,228380.0,107805.0,208825.0,389091.0
2,Bacteroides sp. HF-5141,67049.0,79152.0,5910.0,14776.0,43027.0,55889.0,8493.0,52283.0,29528.0
3,Bacteroides sp. HF-5287,44555.0,36336.0,30711.0,18380.0,182951.0,25675.0,24111.0,159820.0,36717.0
4,Bacteroides sp. CBA7301,28665.0,24532.0,10113.0,11447.0,45735.0,20154.0,8657.0,21988.0,14458.0


In [30]:
dfs_merged.to_csv("Bracken_Noro_baselines.csv", index=None)